In [2]:
%%capture
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [3]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-09 16:43:58.055311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 16:43:58.055467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 16:43:58.192615: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login
login('')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from datasets import load_dataset, concatenate_datasets
import random
from datasets import Dataset, DatasetDict


dataset = load_dataset("Angelectronic/ViMMRC")
dataset

Generating train split:   0%|          | 0/1936 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/551 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/296 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 1936
    })
    test: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 551
    })
    dev: Dataset({
        features: ['context', 'question', 'options', 'answers'],
        num_rows: 296
    })
})

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = '',
)
tokenizer.padding_side = "left"

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [7]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
END_TURN_TOKEN = '<end_of_turn>'
BOS_TOKEN = tokenizer.bos_token

def formatting_prompts_func(examples):
    texts = []
    for i in range(len(examples['question'])):
        option_prompt = ""
        for j in range(len(examples['options'][i])):
            if j == 0:
                option_prompt += "A. " + examples['options'][i][j] + "\n"
            elif j == 1:
                option_prompt += "B. " + examples['options'][i][j] + "\n"
            elif j == 2:
                option_prompt += "C. " + examples['options'][i][j] + "\n"
            elif j == 3:
                option_prompt += "D. " + examples['options'][i][j] + "\n"

        chat = [
            {"role": "user", "content": "Use the following reference to answer the given multiple choice-question:\nReference: " + examples['context'][i] + "\nQuestion: " + examples['question'][i] + "\n" + option_prompt},
            {"role": "system", "content": examples['answers'][i]}
        ]
        texts.append(tokenizer.apply_chat_template(chat, tokenize=False) + EOS_TOKEN)

    texts = [text.split(BOS_TOKEN)[-1] for text in texts]
    return {"text": texts}
pass
dataset['train'] = dataset['train'].map(formatting_prompts_func, batched=True)
dataset['test'] = dataset['test'].map(formatting_prompts_func, batched=True)

collator = DataCollatorForCompletionOnlyLM(instruction_template="<start_of_turn>user", response_template="<start_of_turn>model", tokenizer=tokenizer, mlm=False)

print(dataset['train'][-1]['text'])

Map:   0%|          | 0/1936 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

<|start_header_id|>user<|end_header_id|>

Use the following reference to answer the given multiple choice-question:
Reference: Khách đến xã Trịnh Tường, huyện Bát Xát, tỉnh Lào Cai  sẽ không khỏi ngỡ ngàng thấy một dòng mương ngoằn ngoèo vắt ngang những đồi cao. Dân bản gọi dòng mương ấy là con nước ông Lìn. Để thay đổi tập quán làm lúa nương, ông Phàn Phù Lìn, người Dao ở thôn Phìn Ngan đã lần mò cả tháng trong rừng tìm nguồn nước. Nhưng tìm được nguồn nước rồi, mọi người vẫn không tin có thể dẫn nước về. Ông cùng vợ con đào suốt một năm trời được gần bốn cây số mương xuyên đồi dẫn nước từ rừng già về thôn, trồng một héc ta lúa nước để bà con tin. Rồi ông vận động mọi người cùng mở rộng con mương, vỡ thêm đất hoang trồng lúa.

Con nước nhỏ đã làm thay đổi tập quán canh tác và cuộc sống của trên 50 hộ trong thôn. Những nương lúa quanh năm khát nước được thay dần bằng ruộng bậc thang. Những giống lúa lai cao sản được ông Lìn đưa về vận động bà con trồng cấy, nhờ vậy mà cả thôn không còn

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

END_TURN_TOKEN = '<|eot_id|>'
BOS_TOKEN = tokenizer.bos_token
EOS_TOKEN = tokenizer.eos_token

labels = [dataset['test'][i]['answers'] for i in range(len(dataset['test']))]
preds = []
preds1 = []
FastLanguageModel.for_inference(model)
for i in tqdm(range(len(dataset['test']))):
    option_prompt = ""
    for j in range(len(dataset['test'][i]['options'])):
        if j == 0:
            option_prompt += "A. " + dataset['test'][i]['options'][j] + "\n"
        elif j == 1:
            option_prompt += "B. " + dataset['test'][i]['options'][j] + "\n"
        elif j == 2:
            option_prompt += "C. " + dataset['test'][i]['options'][j] + "\n"
        elif j == 3:
            option_prompt += "D. " + dataset['test'][i]['options'][j] + "\n"

    chat = [
        {"role": "user", "content": "Use the following reference to answer the given multiple choice-question, respond only with the letter before the correct answer (A, B, C or D) and do not add anything else :\nReference: " + dataset['test'][i]['context'] + "\nQuestion: " + dataset['test'][i]['question'] + "\n" + option_prompt},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    prompt = prompt.split(BOS_TOKEN)[-1]
    inputs = tokenizer(
        prompt,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=256, use_cache = True, do_sample = True, eos_token_id=model.config.eos_token_id, pad_token_id=tokenizer.eos_token_id)

#     logits = outputs.logits
#     token_ids = torch.argmax(logits, dim=-1)
    
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=False)
    completion = prediction.split('<|start_header_id|>assistant<|end_header_id|>')[-1].split(END_TURN_TOKEN)[0].strip()
    preds.append(completion)

print(accuracy_score(labels, preds))

In [ ]:
print(preds[0])